In [4]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt

In [5]:
data1=[]
data1=pd.read_csv("ese527_spring/偿债能力/FI_T1.csv")
columnss=['Stkcd','Accper','Typrep','Current_Ratio','Liquidity_Ratio','Cash_Ratio','Cash_Flow_Interest_Coverage_Ratio','Asset-liability_Ratio','Equity_Ratio']
data1.columns=columnss

In [6]:
data2=[]
data2=pd.read_csv("ese527_spring/发展能力/FI_T8.csv")
columnss=['Stkcd','Accper','Typrep','Total_Assets_Growth_Rate','Net_Profit_Growth_Rate','Comprehensive_Income_Growth_Rate','Sustainable_Growth_Rate','Owner_s_Equity_Growth_Rate']
data2.columns=columnss

In [7]:
data3=[]
data3=pd.read_csv("ese527_spring/风险水平/FI_T7.csv")
columnss=['Stkcd','Accper','Typrep','Financial_Leverage','Operating_Leverage']
data3.columns=columnss

In [8]:
data4=[]
data4=pd.read_csv("ese527_spring/经营能力/FI_T4.csv")
columnss=['Stkcd','Accper','Typrep','Accounts_Receivable_Turnover_Ratio','Inventory_Turnover_Ratio','Working_Capital_Turnover_Ratio','Cash_and_Cash_Equivalents_Turnover_Ratio','Current_Assets_Turnover_Ratio','Fixed_Assets_Turnover_Ratio','Total_Asset_Turnover_Ratio']
data4.columns=columnss

In [9]:
data5=[]
data5=pd.read_csv("ese527_spring/每股指标/FI_T9.csv")
columnss=['Stkcd','Accper','Typrep','EPS']
data5.columns=columnss

In [10]:
data6=[]
data6=pd.read_csv("ese527_spring/现金流分析/FI_T6.csv")
columnss=['Stkcd','Accper','Typrep','Operating_Profit_Net_Cash_Content','Total_Cash_Recovery','Operational_Index']
data6.columns=columnss

In [11]:
data7=[]
data7=pd.read_csv("ese527_spring/盈利能力/FI_T5.csv")
columnss=['Stkcd','Accper','Typrep','Return_On_Assets','Return_On_Net_Assets','EBIT','Operating_Profit_Margin','Cost_Profit_Margin','Cash-To-Profit_Ratio','Return_On_Investment']
data7.columns=columnss

In [12]:
# 合并数据
data=data1
# data = data.merge(data1, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data2, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data3, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data4, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data6, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data7, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
data = data.merge(data5, left_on=['Stkcd','Accper','Typrep'], right_on=['Stkcd','Accper','Typrep'])
# data = data.drop("ShortName", axis=1) # shortname 是公司名称不需要

In [13]:
# 数据重新排序，去掉单独母公司报表，加入一行EPS
data = data.sort_values(by=['Stkcd','Accper','Typrep'])
data.reset_index(drop=True, inplace=True)
#data=data.dropna()
data['Future_EPS']=np.nan
data=data.drop(data[data['Typrep']=='B'].index)
data=data.drop(['Typrep'],axis=1)

In [14]:
tickers=pd.read_csv("ticker.csv", header= None)
tickers

,0
0,OEDV
1,AAPL
2,BAC
3,AMZN
4,T
...,...
5830,BTOW
5831,BPE.MI
5832,BNG.TO
5833,BJWTF


In [15]:
global count
count=0


def summ(x):
    global count
    global dataoutout
    x=x.reset_index(drop=True)
    for i in range(len(x['Stkcd'])):
        x.iloc[i,0]=tickers.iloc[count,0]
    count=count+1
    dataoutout.append(x)

In [16]:
dataoutout=[]
data.groupby('Stkcd',group_keys=False).apply(summ)

""


In [17]:
data_out=pd.concat(dataoutout)
data_out=data_out.reset_index(drop=True)
data_out

,Stkcd,Accper,Current_Ratio,Liquidity_Ratio,Cash_Ratio,Cash_Flow_Interest_Coverage_Ratio,Asset-liability_Ratio,Equity_Ratio,Total_Assets_Growth_Rate,Net_Profit_Growth_Rate,...,Operational_Index,Return_On_Assets,Return_On_Net_Assets,EBIT,Operating_Profit_Margin,Cost_Profit_Margin,Cash-To-Profit_Ratio,Return_On_Investment,EPS,Future_EPS
0,OEDV,2016-03-31,NaN,NaN,NaN,NaN,0.930042,13.294233,0.163304,0.081187,...,11.265539,0.003095,0.034870,8.029000e+09,0.301618,NaN,7.381741,0.002443,0.424278,NaN
1,OEDV,2016-06-30,NaN,NaN,NaN,NaN,0.931560,13.611283,0.089661,0.061027,...,-3.962048,0.006087,0.069604,1.615400e+10,0.306461,NaN,-2.055095,0.004741,0.716016,NaN
2,OEDV,2016-09-30,NaN,NaN,NaN,NaN,0.928982,13.080947,0.073941,0.055186,...,-9.483011,0.009271,0.104073,2.456000e+10,0.311390,NaN,-6.227280,0.008040,1.089342,NaN
3,AAPL,2016-03-31,1.291051,0.387376,0.109543,-44.356706,0.789403,3.748416,0.252446,0.376418,...,-13.014385,0.003038,0.009088,1.929084e+09,0.112694,0.139687,-6.357096,0.009832,0.109533,NaN
4,AAPL,2016-06-30,1.245219,0.406726,0.139070,60.397403,0.806019,4.155141,0.326610,0.044944,...,3.631853,0.015726,0.051694,1.040747e+10,0.132423,0.166989,2.584802,0.030460,0.635818,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80214,STRP,2020-09-30,3.279370,3.279370,0.538087,2.232341,0.514825,1.061111,0.000138,0.039218,...,7.286979,0.039270,0.034670,3.996302e+07,0.215353,0.255361,2.726799,0.031628,0.045780,NaN
80215,STRP,2020-12-31,2.991174,2.991174,0.189276,1.853899,0.505506,1.022270,-0.008993,-0.137421,...,1.221283,0.052614,0.044811,5.330533e+07,0.223599,0.261335,2.409628,0.045190,0.059906,NaN
80216,STRP,2021-03-31,3.115942,3.115942,0.292632,-0.220122,0.502220,1.008921,0.016049,2.879847,...,NaN,0.014636,0.016635,1.483519e+07,0.300002,0.397978,-0.170217,0.014391,NaN,NaN
80217,STRP,2021-06-30,3.743460,3.743460,0.034283,-0.057328,0.491683,0.967277,-0.003038,0.343907,...,NaN,0.028909,0.029463,2.918387e+07,0.276184,0.364456,-0.050729,0.024851,NaN,NaN


In [18]:
for i in range(3):
    data_out.iloc[i,0]='SP500'

In [19]:
##条件判断测试
#data2. == data2.TradingDate[i]+datetime.timedelta(days=21)]
#data2[data2['TradingDate'] == data2.TradingDate[1]+datetime.timedelta(days=1000)]
#data2.TradingDate[1]+datetime.timedelta(days=1000) in data2['TradingDate']
data_out.to_csv("data.csv", index=False)
data = pd.read_csv("data.csv",dtype={'Symbol': np.str},parse_dates=['Accper'])

`np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
